# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor

## Part 1 - Data Preprocessing

### Importing the dataset

In [2]:
dataset = pd.read_excel('Dataset3.xlsx')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546 entries, 0 to 545
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Charge_type        546 non-null    object 
 1   Charge_size        546 non-null    float64
 2   Standoff_distance  546 non-null    float64
 3   Incident_pressure  546 non-null    float64
dtypes: float64(3), object(1)
memory usage: 17.2+ KB


In [3]:
# convert categorical variable into dummy variables
dataset = pd.get_dummies(dataset, columns=['Charge_type'])
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546 entries, 0 to 545
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Charge_size        546 non-null    float64
 1   Standoff_distance  546 non-null    float64
 2   Incident_pressure  546 non-null    float64
 3   Charge_type_CompB  546 non-null    uint8  
 4   Charge_type_TNT    546 non-null    uint8  
dtypes: float64(3), uint8(2)
memory usage: 14.0 KB


In [4]:
y = dataset['Incident_pressure']
X = dataset.drop('Incident_pressure', axis=1)
print(X.shape, y.shape)

(546, 4) (546,)


In [5]:
# convert to numpy array
X = np.array(X)
y = np.array(y)

### Splitting the dataset into the Training set and Test set

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

### Hyperparameter tuning - layers, neurons, activation function

In [7]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor as KR
import math
def FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes):
    layers = []
    
    nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)
    nodes = first_layer_nodes
    for i in range(1, n_layers+1):
        layers.append(math.ceil(nodes))
        nodes = nodes + nodes_increment
    
    return layers

In [8]:
from tensorflow.keras.optimizers import Adam
def create_model(n_layers, first_layer_nodes, last_layer_nodes, activation_func):
    model = Sequential()
    n_nodes = FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes)
    for i in range(1, n_layers):
        if i==1:
            model.add(Dense(units = first_layer_nodes,  input_shape=(X_train.shape[1],), activation=activation_func))
        else:
            model.add(Dense(n_nodes[i-1], activation=activation_func))
            
    #Finally, the output layer should have a single node in binary classification
    model.add(Dense(1, activation='linear'))
    opt = Adam(learning_rate=0.001)
    model.compile(optimizer = opt, loss = 'mean_squared_error', metrics = ['mae'])
    return model

##Wrap model into scikit-learn
model = KR(build_fn=create_model, epochs = 500, batch_size = 50)

C:\Users\Chathura Gamage\AppData\Local\Temp\ipykernel_9876\2480470563.py:18: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KR(build_fn=create_model, epochs = 500, batch_size = 50)


In [9]:
activation_funcs = ['relu', 'softplus', 'leaky_relu'] 
param_grid = dict(n_layers=[2,3], first_layer_nodes = [100, 120, 140, 150], last_layer_nodes = [8,10, 30, 50],  activation_func = activation_funcs)
grid = GridSearchCV(estimator = model, param_grid = param_grid,n_jobs=-1, cv=5, scoring = 'r2')
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/500
8/8 [==============================] - 1s 2ms/step - loss: 1331212.3750 - mae: 438.2138
Epoch 2/500
8/8 [==============================] - 0s 2ms/step - loss: 1326378.1250 - mae: 432.5815
Epoch 3/500
8/8 [==============================] - 0s 2ms/step - loss: 1321128.8750 - mae: 426.3721
Epoch 4/500
8/8 [==============================] - 0s 2ms/step - loss: 1314396.0000 - mae: 418.1515
Epoch 5/500
8/8 [==============================] - 0s 2ms/step - loss: 1306687.6250 - mae: 406.9132
Epoch 6/500
8/8 [==============================] - 0s 2ms/step - loss: 1296511.0000 - mae: 391.5524
Epoch 7/500
8/8 [==============================] - 0s 2ms/step - loss: 1283306.5000 - mae: 372.7118
Epoch 8/500
8/8 [==============================] - 0s 2ms/step - loss: 1269392.8750 - mae: 354.9307
Epoch 9/500
8/8 [==============================] - 0s 2ms/step - loss: 1252987.8750 - mae: 349.9687
Epoch 10/500
8/8 [==============================] - 0s 2ms/step - loss: 1236668.1250 - mae: 356.1588

### Hyperparameter tuning - batch size, epoch, optimizer, learning rate

In [10]:
# Function to create model, required for KerasRegressor
def create_model(optimizer):
 # create model
 model = Sequential()
 model.add(Dense(units=200, input_shape=(X_train.shape[1],), activation='relu'))
 model.add(Dense(units=125, activation='relu'))
 model.add(Dense(units=50, activation='relu'))
 model.add(Dense(units=1, activation='linear'))
 # Compile model
 model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['mae'])
 return model

In [11]:
# create model
model = KerasRegressor(model=create_model)

# define the grid search parameters
batch_size = [50, 60, 80]
epochs = [300, 400, 500]
optimizer = ['RMSprop', 'Adam', 'Adamax', 'Nadam']
learn_rate = [0.001, 0.01, 0.1]

# gridsearch
param_grid = dict(batch_size=batch_size, epochs=epochs, model__optimizer=optimizer, optimizer__learning_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, scoring = 'r2')
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/500
8/8 [==============================] - 0s 3ms/step - loss: 1254501.8750 - mae: 424.3489
Epoch 2/500
8/8 [==============================] - 0s 3ms/step - loss: 1247393.6250 - mae: 415.2352
Epoch 3/500
8/8 [==============================] - 0s 4ms/step - loss: 1236023.5000 - mae: 399.7627
Epoch 4/500
8/8 [==============================] - 0s 4ms/step - loss: 1218660.5000 - mae: 372.0125
Epoch 5/500
8/8 [==============================] - 0s 5ms/step - loss: 1192174.5000 - mae: 338.8174
Epoch 6/500
8/8 [==============================] - 0s 3ms/step - loss: 1159819.6250 - mae: 351.3945
Epoch 7/500
8/8 [==============================] - 0s 3ms/step - loss: 1126903.7500 - mae: 378.8437
Epoch 8/500
8/8 [==============================] - 0s 4ms/step - loss: 1108720.1250 - mae: 413.6395
Epoch 9/500
8/8 [==============================] - 0s 4ms/step - loss: 1081422.2500 - mae: 435.3830
Epoch 10/500
8/8 [==============================] - 0s 4ms/step - loss: 1061987.5000 - mae: 441.2654